In [1]:
import numpy as np
import keras
import matplotlib.pyplot as plt
import struct

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
train_data_fileName="train-images.idx3-ubyte"
train_label_fileName="train-labels.idx1-ubyte"
test_data_fileName="t10k-images.idx3-ubyte"
test_label_fileName="t10k-labels.idx1-ubyte"

In [3]:
def readData(fileName):
    with open(fileName,'rb') as obj:
        data=obj.read(16)
        magic,N,n_row,n_col=struct.unpack('!4i',data)
        images=np.zeros((N,n_row,n_col))
        for i in range(N):
            data=obj.read(n_row*n_col)
            images[i]=np.array(struct.unpack('!'+str(n_row*n_col)+'B',data)).reshape((n_row,n_col))
    return images
def readLabels(fileName):
    with open(fileName,'rb') as obj:
        data=obj.read(8)
        magic,N=struct.unpack('!2i',data)
        data=obj.read(N)
        labels=np.array(struct.unpack('!'+str(N)+'B',data))
    return labels

In [4]:
train_data=readData(train_data_fileName)
N,n_row,n_col=train_data.shape
train_data=train_data.reshape((N,n_row,n_col,1))
train_labels=np.eye(10)[readLabels(train_label_fileName)]

test_data=readData(test_data_fileName)
N,n_row,n_col=test_data.shape
test_data=test_data.reshape((N,n_row,n_col,1))
test_labels=np.eye(10)[readLabels(test_label_fileName)]


In [53]:
def Model(input_size=(28,28,1)):
    X_Input=keras.layers.Input(input_size)
    X=keras.layers.Conv2D(6,kernel_size=(5,5),strides=(1,1),activation='relu',name='conv_1',padding='same',kernel_initializer='random_uniform')(X_Input)
    X=keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),name='pool_2')(X)
    X=keras.layers.Conv2D(16,kernel_size=(5,5),strides=(1,1),activation='relu',name='conv_3',kernel_initializer='random_uniform')(X)
    X=keras.layers.MaxPool2D(pool_size=(2,2),strides=(2,2),name='pool_4')(X)
    X=keras.layers.Flatten(name='flat_5')(X)
    X=keras.layers.Dense(120,activation='relu',name='fc_6',kernel_initializer='random_uniform')(X)
    X=keras.layers.Dense(84,activation='relu',name='fc_7',kernel_initializer='random_uniform')(X)
    X=keras.layers.Dense(10,activation='softmax',name='output_8',kernel_initializer='random_uniform')(X)
    model=keras.models.Model(inputs=X_Input,outputs=X,name='myModel')
    return model
   

In [80]:
model=Model()
model.compile(metrics=['accuracy'],optimizer='sgd',loss='categorical_crossentropy')

In [81]:
checkpoint_best = keras.callbacks.ModelCheckpoint("mnist_sgd.h5", monitor='val_acc',verbose=1,save_best_only=False)

hist=model.fit(train_data,train_labels,
         batch_size=128,
         epochs=10,
         shuffle=True,
         validation_data=(test_data,test_labels),
         callbacks=[checkpoint_best])


Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 5s 80us/step - loss: 0.4018 - acc: 0.8724 - val_loss: 0.1203 - val_acc: 0.9621

Epoch 00001: saving model to mnist_sgd.h5
Epoch 2/10
60000/60000 [==============================] - 3s 53us/step - loss: 0.0964 - acc: 0.9694 - val_loss: 0.0818 - val_acc: 0.9754

Epoch 00002: saving model to mnist_sgd.h5
Epoch 3/10
60000/60000 [==============================] - 3s 53us/step - loss: 0.0690 - acc: 0.9785 - val_loss: 0.0604 - val_acc: 0.9810

Epoch 00003: saving model to mnist_sgd.h5
Epoch 4/10
60000/60000 [==============================] - 3s 53us/step - loss: 0.0547 - acc: 0.9827 - val_loss: 0.0687 - val_acc: 0.9777

Epoch 00004: saving model to mnist_sgd.h5
Epoch 5/10
60000/60000 [==============================] - 3s 54us/step - loss: 0.0470 - acc: 0.9849 - val_loss: 0.0515 - val_acc: 0.9830

Epoch 00005: saving model to mnist_sgd.h5
Epoch 6/10
60000/60000 [==========================

In [67]:
print(hist.history)

{'val_loss': [0.07795129197500646, 0.04765106523782015, 0.04806212450303137, 0.04508610233180225, 0.0424912239741534, 0.034911226649861786, 0.040264245296875015, 0.04722756129315312, 0.038046044643456116, 0.04749801207685377], 'val_acc': [0.9741, 0.9854, 0.9848, 0.9862, 0.9864, 0.9882, 0.9871, 0.9867, 0.9886, 0.9875], 'loss': [0.2132067693968614, 0.06258009854356447, 0.046084610040485856, 0.0370858515838782, 0.03182641693154971, 0.028507591164360444, 0.023365556684012214, 0.020272714842204005, 0.019634612095169722, 0.017574439414512987], 'acc': [0.9324666666984558, 0.9804000000317892, 0.9859499999682109, 0.9886, 0.9899166666666667, 0.9908333333333333, 0.99285, 0.9935, 0.9937499999682109, 0.9941166666666666]}
